<a href="https://colab.research.google.com/github/tarupathak30/rag-astronomy-chatbot-/blob/main/exoplaents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sentence-transformers faiss-cpu transformers langchain_community
!pip install -q astroquery


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.5 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.
   ━━━━━━━

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=eb91f60412598eafa5f2cb495fd7f7a84fedde4609235820c2dbab06808e50f5
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [ ]:
import wikipedia

page = wikipedia.page("Brown dwarf")


In [ ]:
print(page.content[:500])


Brown dwarfs are substellar objects that have more mass than the biggest gas giant planets, but less than the least massive main-sequence stars.  Their mass is approximately 13 to 80 times that of Jupiter (MJ)—not big enough to sustain nuclear fusion of hydrogen into helium in their cores, but massive enough to emit some light and heat from the fusion of deuterium, 2H, an isotope of hydrogen with a neutron as well as a proton, that can undergo fusion at lower temperatures.  The most massive ones


In [ ]:
import wikipedia

keywords = [
    "Brown dwarf",
    "Gas giant",
    "Planet",
    "Star",
    "Main sequence",
    "Exoplanet",
    "Radial velocity",
    "Transit method",
    "Direct imaging",
    "Neutron star",
    "White dwarf",
    "Galaxy",
    "Dark matter",
    "Dark energy",
    "Supernova",
    "Black hole",
    "Protoplanetary disk",
    "Astrobiology"
]

concept_texts = []

for topic in keywords:
    try:
        page = wikipedia.page(topic)
        if(topic == "Brown dwarf"):
          print(page)
        concept_texts.append((topic, page.content))
    except:
        pass


<WikipediaPage 'Brown dwarf'>


/usr/local/lib/python3.12/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.12/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
# ============================
# 🔭 2. IMPORTS
# ============================
import pandas as pd
import pickle
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

# ============================
# 🔭 3. DOWNLOAD EXOPLANET DATA
# ============================
exo = NasaExoplanetArchive.query_criteria(table="pscomppars")
exoplanets = exo.to_pandas()
exoplanets.fillna("", inplace=True)
exoplanets.to_csv("exoplanets.csv", index=False)

print("Downloaded exoplanet rows:", len(exoplanets))

# ============================
# 🔭 4. BUILD DOCUMENTS
# ============================


/tmp/ipython-input-3209938367.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  exoplanets.fillna("", inplace=True)


Downloaded exoplanet rows: 6045


In [ ]:
from langchain.docstore.document import Document

docs = []

# Group by 'hostname'
grouped = exo.group_by("hostname")

for group in grouped.groups:  # each group is a QTable subset
    star = group['hostname'][0]  # all rows in this group have the same host
    context_text = ""

    for row in group:  # iterate rows in the QTable group
        context_text += (
            f"planet_name: {row['pl_name']}\n"
            f"orbital_period_days: {row['pl_orbper']}\n"
            f"mass_earth: {row['pl_bmasse']}\n"
            f"radius_earth: {row['pl_rade']}\n\n"
        )

    docs.append(
        Document(
            page_content=context_text,
            metadata={"host_star": star}
        )
    )

print(f"Built {len(docs)} system-level documents")


UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.") [astropy.table.table]
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.")
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.") [astropy.table.table]
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.")
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.") [astropy.table.table]
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.")
UnitTypeError("MaskedQuantity instances require normal units, not <class 'astropy.units.function.logarithmic.DexUnit'> instances.") [astropy.t

In [ ]:
# ----------------------------
# 1️⃣ Load reranker
# ----------------------------
from sentence_transformers import CrossEncoder
reranker = CrossEncoder("BAAI/bge-reranker-base")  # CPU or GPU

# ----------------------------
# 2️⃣ Load LLM + tokenizer
# ----------------------------
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"

tok = AutoTokenizer.from_pretrained(MODEL_ID)
llm = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float32,
    device_map="auto"
)
device = next(llm.parameters()).device
print("LLM and reranker loaded on:", device)


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import pickle

# Use LangChain wrapper
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

texts = [d.page_content for d in docs]
metadatas = [d.metadata for d in docs]

index = FAISS.from_texts(
    texts=texts,
    embedding=embedder,
    metadatas=metadatas
)

index.save_local("faiss_index")

# Save docs separately
with open("docs.pkl", "wb") as f:
    pickle.dump(docs, f)

print("✅ FAISS + docs.pkl saved successfully.")


In [ ]:
concept_docs = []

for title, content in concept_texts:
    concept_docs.append(Document(
        page_content=content,
        metadata={"source": title}
    ))


In [ ]:
all_docs = docs + concept_docs
texts = [d.page_content for d in all_docs]
metadatas = [d.metadata for d in all_docs]

index = FAISS.from_texts(
    texts=texts,
    embedding=embedder,
    metadatas=metadatas
)

index.save_local("faiss_index")
pickle.dump(all_docs, open("docs.pkl","wb"))


In [ ]:
# ============================
# 🔭 7. RERANK FUNCTION
# ============================
def rerank_docs(query, docs):
    """
    Rerank candidate docs using CrossEncoder and return the top doc content.
    """
    texts = [d.page_content for d in docs]
    pairs = [(query, t) for t in texts]
    scores = reranker.predict(pairs)  # returns numpy array
    best_idx = int(scores.argmax())
    return docs[best_idx].page_content

In [ ]:
def answer_query(query):
  # adding diversity in the document selection
    # 1. semantic search
    hits = index.max_marginal_relevance_search(query, k=10, fetch_k=30)
    if not hits:
        return "No data available."

    # 2. rerank
    context = rerank_docs(query, hits)

    # 3. build strict prompt
    prompt = f"""
    You are AstroBuddy, a friendly astronomy nerd who loves exoplanets, stars, and exoplanet systems.
You explain things clearly, logically, and with a touch of humor or nerdiness when appropriate.
ALWAYS follow these rules:
1. You MUST answer using ONLY the text in <context>.
2. Do NOT add, infer, or guess anything.
3. If the answer is NOT directly stated in <context>, reply EXACTLY with:
   "I cannot answer using only the provided context."
4. When answering, you can phrase things in a friendly or nerdy tone, but you cannot hallucinate facts or numbers.
5. Numbers and units in the <context> are factual and must be used exactly as written.
<context>
{context}
</context>
Question: {query}
Answer clearly and concisely using ONLY the provided context.
"""

    # 4. llm generate
    inputs = tok(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out = llm.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.05,
            pad_token_id=tok.eos_token_id
        )
    answer = tok.decode(out[0], skip_special_tokens=True).strip()


    return answer


In [ ]:

query = "Which exoplanet has the longest orbital period?"

print(answer_query(query))
